In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
train = pd.read_csv(r"C:\Users\Shraddha\Downloads\playground-series-s5e6\train.csv")
test = pd.read_csv(r"C:\Users\Shraddha\Downloads\playground-series-s5e6\test.csv")

In [3]:
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [5]:
y = encoder.fit_transform(train['Fertilizer Name'])

In [6]:
print(y)

[4 4 2 ... 0 3 6]


In [7]:
train_cat = pd.get_dummies(train[['Soil Type','Crop Type']])
test_cat = pd.get_dummies(test[['Soil Type','Crop Type']])

In [8]:
test_cat = test_cat.reindex(columns=train_cat.columns,fill_value=0)

In [9]:
train_num = train[['Temparature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']]
test_num = test[['Temparature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']]

In [10]:
x = pd.concat([train_num,train_cat],axis=1)
x_test_final = pd.concat([test_num,test_cat],axis=1)

In [11]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [12]:
model = DecisionTreeClassifier(max_depth=15,random_state=42)
model.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=15, random_state=42)

In [13]:
probs = model.predict_proba(x_test_final)

In [14]:
top_3 = np.argsort(probs,axis=1)[:,-3:][:,::-1]

In [15]:
top_3_labels = encoder.inverse_transform(top_3.ravel()).reshape(top_3.shape)

In [16]:
top_3_stg = [' '.join(row) for row in top_3_labels]

In [18]:
Optimal_Fertilizers_by_DecisionTree = pd.DataFrame({'id' : test['id'],'Fertilizer Name' : top_3_stg})
Optimal_Fertilizers_by_DecisionTree.to_csv("Optimal_Fertilizers_by_DecisionTree.csv",index=False)
print("Saved")

Saved


In [17]:
def mapk(actual, predicted, k=3):
    score = 0.0
    for act, pred in zip(actual, predicted):
        try:
            rank = pred.index(act)
            if rank < k:
                score += 1.0 / (rank + 1)
        except ValueError:
            pass
    return score / len(actual)

# Convert y_val to original labels
y_val_labels = encoder.inverse_transform(y_test)
val_pred_strings = [[encoder.inverse_transform([i])[0] for i in row] for row in top_3]

val_score = mapk(y_val_labels, val_pred_strings, k=3)
print(f"Validation MAP@3 score: {val_score:.4f}")

Validation MAP@3 score: 0.2685
